In [1]:
import sys
sys.path.insert(0, '/das/work/p17/p17983/')

In [2]:
%matplotlib nbagg
import jungfrau_utils as ju
import numpy as np
import json
import os
import time
from matplotlib import pyplot as plt
from scipy.optimize import curve_fit

import h5py

from alvra_tools.load_data import *
from alvra_tools.channels import *
from built_functions import *

[INFO][corrections] Numba not available, reverting to Numpy


In [3]:
def get_xes_pumped(filename, DIR, DIRBS, roi, channel, ynstamp):
    numstds = 2.5
    minIzero = 0.01
    lin_filter = 0.2
    image_threshold = 1.5
    hot_pixel = 10

    print(DIR + filename + ".JF02T09V02crop.h5")
    print()
    print(DIRBS + filename + ".BSREAD.h5")
    
    images_off, images_on, pulse_ids_off, pulse_ids_on = \
        load_JF_cropped_data_pump(DIR + filename + ".JF02T09V02crop.h5", roi, 50, 50/3, nshots=None)
    (DataFluo_pump, DataFluo_unpump, IzeroFEL_pump, IzeroFEL_unpump, JFIDs_pump, JFIDs_unpump, Energy_pump, Energy_unpump, _, _) = \
        load_PumpProbe_events_withTwoVariables(DIRBS + filename + ".BSREAD.h5", channel_BS_pulse_ids, channel)
    (_, _, _, _, _, _, Laser_pump, Laser_unpump, _, _) = \
        load_PumpProbe_events_withTwoVariables(DIRBS + filename + ".BSREAD.h5", channel_PIPS_fluo, channel)
    
    Energy_pump = [x for x in Energy_pump if (np.abs(x) > 0)]
    Energy_eV_pump = np.mean(Energy_pump)
    
    Energy_unpump = [x for x in Energy_unpump if (np.abs(x) > 0)]
    Energy_eV_unpump = np.mean(Energy_unpump)
    
    idcheck = [a==b and c==d for a,b,c,d in zip(pulse_ids_on, JFIDs_pump, pulse_ids_off, JFIDs_unpump)]
    IDCheck = sum([int(x) for x in idcheck])
    
    if IDCheck:
        print("passed ID check")
    else:
        print("failed ID check")   
    
    IzeroMedian = np.median(np.concatenate([IzeroFEL_pump, IzeroFEL_unpump]))
    IzeroSTD = np.std(np.concatenate([IzeroFEL_pump, IzeroFEL_unpump]))
    
    print('numon')
    print(IzeroFEL_pump.shape[0])
    
    print('numoff')
    print(IzeroFEL_unpump.shape[0])
    
    print('number of frames')
    print(images_on.shape[0])
    print(images_off.shape[0])

    images_good_on = images_on
    images_good_off = images_off

    images_thr_on = images_good_on.copy()
    images_thr_on[images_good_on < image_threshold] = 0
    images_thr_on[images_good_on > hot_pixel] = 0
    images_thr_on[np.isnan(images_thr_on)] = 0
    
    images_thr_off = images_good_off.copy()
    images_thr_off[images_good_off < image_threshold] = 0
    images_thr_off[images_good_off > hot_pixel] = 0
    images_thr_off[np.isnan(images_thr_off)] = 0
    
    print('num nan')
    print(sum(sum(sum(np.isnan(images_thr_on)))))
    
    print('Izero')
    print(np.mean(IzeroFEL_unpump))
    print(np.mean(IzeroFEL_pump))
    print(np.max(IzeroFEL_unpump))
    print(np.min(IzeroFEL_unpump))
    
    IzeroFEL_pump, IzeroFEL_unpump, DataFluo_pump, DataFluo_unpump, condFinalPump, condFinalUnPump = \
        filterData(IzeroFEL_pump, IzeroFEL_unpump, DataFluo_pump, DataFluo_unpump, numstds, minIzero, lin_filter)
    

    
    print(images_thr_on.shape)
    print(images_thr_on.shape)
    
    images_thr_on = images_thr_on[condFinalPump]
    images_thr_off = images_thr_off[condFinalUnPump]

    print('number of surviving frames')
    print(images_thr_on.shape[0])
    print(images_thr_off.shape[0])
    
    XES_on = images_thr_on.sum(axis=0)/images_thr_on.shape[0]
    XES_off = images_thr_off.sum(axis=0)/images_thr_off.shape[0]
    
    XES_on[np.isnan(XES_on)] = 0
    XES_off[np.isnan(XES_off)] = 0
    
    print('num nan')
    print(sum(sum(np.isnan(XES_on))))
    
    stamp = make_bar_stamp(XES_on.shape[1],XES_on.shape[0])
    
    print(XES_on.shape)
    print(stamp.shape)
    if ynstamp:

        XES_on = XES_on * stamp
        XES_off = XES_off * stamp
        
    FloFEL_pump_total = np.mean(DataFluo_pump)/np.mean(IzeroFEL_pump)
    FloFEL_unpump_total = np.mean(DataFluo_unpump)/np.mean(IzeroFEL_unpump)
    
    return XES_on, XES_off, Energy_eV_pump, Energy_eV_unpump, FloFEL_pump_total, FloFEL_unpump_total


In [4]:
def make_bar_stamp(dimension_x, dimension_y):

    X,Y = np.meshgrid(np.linspace(0,dimension_x-1,dimension_x),np.linspace(0,dimension_y-1,dimension_y))

    Z = np.zeros([dimension_y,dimension_x])+1

    condition = np.logical_and(Y<260, Y>253)
    Z[condition] = 0
    
    Z[Y>320] = 0
    Z[Y < 200] = 0

    condition = np.logical_not(np.logical_and(X<200, X>130))
    Z[condition] = 0

    #plt.figure()
    #plt.imshow(Z)
    
    #plt.colorbar()
    
    return Z

Run for RIXS

In [5]:
def make_rixs_planes(scannum, base, numfiles):
    
    for jj in range(len(scannum)):
        
        filename_base = base + '%02d' % scannum[jj] + "_step00"
        scan_name = base + '%02d' % scannum[jj]
        DIR = "/das/work/p17/p17983/cropped_data/scan_data/" + scan_name + "/"
        DIRBS = "/sf/alvra/data/p17983/raw/scan_data/" + scan_name + "/"


    #     filename_base = "RuBpy3_monoscan_0" + '%02d' % scannum[jj] + "_step00"
    #     scan_name = "RuBpy3_monoscan_0" + '%02d' % scannum[jj]
    #     DIR = "/das/work/p17/p17983/cropped_data/scan_data/" + scan_name + "/"
    #     DIRBS = "/sf/alvra/data/p17983/raw/scan_data/" + scan_name + "/"
    #     numfiles = 35+1

        eV_on = np.empty(numfiles)
        eV_off = np.empty(numfiles)

        fluo_Pump = np.empty(numfiles)
        fluo_Unpump = np.empty(numfiles)


        for ii in range(numfiles):

            XES_on, XES_off, Energy_eV_pump, Energy_eV_unpump, fluo_pump, fluo_unpump = \
                get_xes_pumped(filename_base + '%02d' % (ii+0), DIR, DIRBS, 2, channel_energy, True)

            if False:

                plt.figure()
                plt.subplot(1,2,1)
                plt.imshow(XES_on, vmax = 0.001)
                plt.colorbar()
                plt.subplot(1,2,2)
                plt.imshow(XES_off, vmax = 0.001)
                plt.colorbar()
                plt.tight_layout()


            if ii == 0 & jj == 0:
                rixs_on = XES_on.sum(axis=0)
                rixs_off = XES_off.sum(axis=0)
            else:
                rixs_on = np.vstack((rixs_on,XES_on.sum(axis=0)))
                rixs_off = np.vstack((rixs_off,XES_off.sum(axis=0)))

            eV_on[ii] = Energy_eV_pump
            eV_off[ii] = Energy_eV_unpump
            fluo_Pump[ii] = fluo_pump
            fluo_Unpump[ii] = fluo_unpump

        if jj == 0:

            RIXS_on = rixs_on
            RIXS_off = rixs_off
            Fluo_pump = fluo_Pump
            Fluo_unpump = fluo_Unpump

        else:

            RIXS_on = RIXS_on + rixs_on
            RIXS_off = RIXS_off + rixs_off
            Fluo_pump = Fluo_pump + fluo_Pump
            Fluo_unpump = Fluo_unpump + fluo_Unpump

    return eV_on, eV_off, RIXS_on, RIXS_off, Fluo_pump, Fluo_unpump

In [6]:
def save_data(save_name_base, scannum):
    save_name = save_name_base
    for jj in range(0, len(scannum)):
        save_name = save_name + '%02d' % scannum[jj] + '_'
    
    np.save(save_name + 'eV_on.npy', eV_on)
    print(save_name + 'eV_on.npy' + ' saved!')
    np.save(save_name + 'eV_off.npy', eV_off)
    print(save_name + 'eV_off.npy' + ' saved!')
    np.save(save_name + 'RIXS_on.npy', RIXS_on)
    print(save_name + 'RIXS_on.npy' + ' saved!')
    np.save(save_name + 'RIXS_off.npy', RIXS_off)
    print(save_name + 'RIXS_off.npy' + ' saved!')
    np.save(save_name + 'Fluo_pump.npy', Fluo_pump)
    print(save_name + 'Fluo_pump.npy' + ' saved!')
    np.save(save_name + 'Fluo_unpump.npy', Fluo_unpump)
    print(save_name + 'Fluo_unpump.npy' + ' saved!')    
   

In [7]:
def load_data(load_name_base, scannum):
    load_name = load_name_base
    
    for jj in range(0, len(scannum)):
        load_name = load_name + '%02d' % scannum[jj] + '_'
    
    eV_on = np.load(load_name + 'eV_on.npy')
    eV_off = np.load(load_name + 'eV_off.npy')
    RIXS_on = np.load(load_name + 'RIXS_on.npy')
    RIXS_off = np.load(load_name + 'RIXS_off.npy')
    Fluo_pump = np.load(load_name + 'Fluo_pump.npy')
    Fluo_unpump = np.load(load_name + 'Fluo_unpump.npy')
    
    return eV_on, eV_off, RIXS_on, RIXS_off, Fluo_pump, Fluo_unpump

In [8]:
saveON = 1

# #---------------  RuDimerACN, 0.6 ps, horz pol
# scannum = [6, 7, 9]
# base = "RuDimerACN_monoscan_0p6ps_0"
# numfiles = 54+1
# SBASE = '/das/work/p17/p17983/Reduced_data/RuDimerACN_0p6ps_scans_'

# #---------------  RuDimerACN, 10 ps, horz pol
# scannum = [5, 8, 10, 13]
# base = "RuDimerACN_monoscan_10ps_0"
# numfiles = 54+1
# SBASE = '/das/work/p17/p17983/Reduced_data/RuDimerACN_10ps_scans_'

#--------------- RuDimerACN, 0.6 ps, vert pol
scannum = [10, 11, 12, 14]
base = "RuDimerACN_monoscan_0p6ps_0"
numfiles = 54+1
SBASE = '/das/work/p17/p17983/Reduced_data/RuDimerACN_0p6ps_scans_'

# #--------------- RuDimerACN, 10 ps, vert pol
# scannum = [15, 16, 17, 18]
# base = "RuDimerACN_monoscan_10ps_0"
# numfiles = 54+1
# SBASE = '/das/work/p17/p17983/Reduced_data/RuDimerACN_10ps_scans_'




eV_on, eV_off, RIXS_on, RIXS_off, Fluo_pump, Fluo_unpump = make_rixs_planes(scannum, base, numfiles)

if saveON:
    save_data(SBASE, scannum)


/das/work/p17/p17983/cropped_data/scan_data/RuDimerACN_monoscan_0p6ps_010/RuDimerACN_monoscan_0p6ps_010_step0000.JF02T09V02crop.h5

/sf/alvra/data/p17983/raw/scan_data/RuDimerACN_monoscan_0p6ps_010/RuDimerACN_monoscan_0p6ps_010_step0000.BSREAD.h5
images_roi2
SLAAR11-LMOT-M451:ENC_1_BS/pulse_id
SLAAR11-LMOT-M451:ENC_1_BS/pulse_id
passed ID check
numon
666
numoff
334
number of frames
666
334
num nan
0
Izero
0.0521055993562687
0.05201152811252006
0.1334538757801056
0.010792821645736694
(666, 512, 300)
(666, 512, 300)
number of surviving frames
649
326
num nan
0
(512, 300)
(512, 300)
/das/work/p17/p17983/cropped_data/scan_data/RuDimerACN_monoscan_0p6ps_010/RuDimerACN_monoscan_0p6ps_010_step0001.JF02T09V02crop.h5

/sf/alvra/data/p17983/raw/scan_data/RuDimerACN_monoscan_0p6ps_010/RuDimerACN_monoscan_0p6ps_010_step0001.BSREAD.h5
images_roi2
SLAAR11-LMOT-M451:ENC_1_BS/pulse_id
SLAAR11-LMOT-M451:ENC_1_BS/pulse_id
passed ID check
numon
666
numoff
334
number of frames
666
334
num nan
0
Izero
0.05

SLAAR11-LMOT-M451:ENC_1_BS/pulse_id
SLAAR11-LMOT-M451:ENC_1_BS/pulse_id
passed ID check
numon
666
numoff
334
number of frames
666
334
num nan
0
Izero
0.053712844048364614
0.0531407858052098
0.135809525847435
0.010915622115135193
(666, 512, 300)
(666, 512, 300)
number of surviving frames
642
329
num nan
0
(512, 300)
(512, 300)
/das/work/p17/p17983/cropped_data/scan_data/RuDimerACN_monoscan_0p6ps_010/RuDimerACN_monoscan_0p6ps_010_step0015.JF02T09V02crop.h5

/sf/alvra/data/p17983/raw/scan_data/RuDimerACN_monoscan_0p6ps_010/RuDimerACN_monoscan_0p6ps_010_step0015.BSREAD.h5
images_roi2
SLAAR11-LMOT-M451:ENC_1_BS/pulse_id
SLAAR11-LMOT-M451:ENC_1_BS/pulse_id
passed ID check
numon
667
numoff
333
number of frames
667
333
num nan
0
Izero
0.05345924551299474
0.051211871008487835
0.1337278038263321
0.008523843251168728
(667, 512, 300)
(667, 512, 300)
number of surviving frames
647
318
num nan
0
(512, 300)
(512, 300)
/das/work/p17/p17983/cropped_data/scan_data/RuDimerACN_monoscan_0p6ps_010/RuDimerAC

SLAAR11-LMOT-M451:ENC_1_BS/pulse_id
SLAAR11-LMOT-M451:ENC_1_BS/pulse_id
passed ID check
numon
667
numoff
333
number of frames
667
333
num nan
0
Izero
0.05420087572283674
0.05530822032721273
0.1571999043226242
0.0019190647872164845
(667, 512, 300)
(667, 512, 300)
number of surviving frames
639
322
num nan
0
(512, 300)
(512, 300)
/das/work/p17/p17983/cropped_data/scan_data/RuDimerACN_monoscan_0p6ps_010/RuDimerACN_monoscan_0p6ps_010_step0029.JF02T09V02crop.h5

/sf/alvra/data/p17983/raw/scan_data/RuDimerACN_monoscan_0p6ps_010/RuDimerACN_monoscan_0p6ps_010_step0029.BSREAD.h5
images_roi2
SLAAR11-LMOT-M451:ENC_1_BS/pulse_id
SLAAR11-LMOT-M451:ENC_1_BS/pulse_id
passed ID check
numon
667
numoff
333
number of frames
667
333
num nan
0
Izero
0.05301206921403472
0.05286738446130343
0.16226768493652344
0.008673095144331455
(667, 512, 300)
(667, 512, 300)
number of surviving frames
650
324
num nan
0
(512, 300)
(512, 300)
/das/work/p17/p17983/cropped_data/scan_data/RuDimerACN_monoscan_0p6ps_010/RuDimer

SLAAR11-LMOT-M451:ENC_1_BS/pulse_id
SLAAR11-LMOT-M451:ENC_1_BS/pulse_id
passed ID check
numon
666
numoff
334
number of frames
666
334
num nan
0
Izero
0.05290274775259735
0.05211548629592796
0.14745637774467468
-0.0007224415894597769
(666, 512, 300)
(666, 512, 300)
number of surviving frames
620
310
num nan
0
(512, 300)
(512, 300)
/das/work/p17/p17983/cropped_data/scan_data/RuDimerACN_monoscan_0p6ps_010/RuDimerACN_monoscan_0p6ps_010_step0043.JF02T09V02crop.h5

/sf/alvra/data/p17983/raw/scan_data/RuDimerACN_monoscan_0p6ps_010/RuDimerACN_monoscan_0p6ps_010_step0043.BSREAD.h5
images_roi2
SLAAR11-LMOT-M451:ENC_1_BS/pulse_id
SLAAR11-LMOT-M451:ENC_1_BS/pulse_id
passed ID check
numon
666
numoff
334
number of frames
666
334
num nan
0
Izero
0.05231230680325468
0.050257329326100275
0.12242843955755234
-0.0002702907077036798
(666, 512, 300)
(666, 512, 300)
number of surviving frames
608
308
num nan
0
(512, 300)
(512, 300)
/das/work/p17/p17983/cropped_data/scan_data/RuDimerACN_monoscan_0p6ps_010/Ru

SLAAR11-LMOT-M451:ENC_1_BS/pulse_id
SLAAR11-LMOT-M451:ENC_1_BS/pulse_id
passed ID check
numon
667
numoff
333
number of frames
667
333
num nan
0
Izero
0.05158608936253367
0.05175843304045727
0.14022187888622284
-7.661402923986316e-05
(667, 512, 300)
(667, 512, 300)
number of surviving frames
637
316
num nan
0
(512, 300)
(512, 300)
/das/work/p17/p17983/cropped_data/scan_data/RuDimerACN_monoscan_0p6ps_011/RuDimerACN_monoscan_0p6ps_011_step0002.JF02T09V02crop.h5

/sf/alvra/data/p17983/raw/scan_data/RuDimerACN_monoscan_0p6ps_011/RuDimerACN_monoscan_0p6ps_011_step0002.BSREAD.h5
images_roi2
SLAAR11-LMOT-M451:ENC_1_BS/pulse_id
SLAAR11-LMOT-M451:ENC_1_BS/pulse_id
passed ID check
numon
667
numoff
333
number of frames
667
333
num nan
0
Izero
0.04489130670753274
0.04709445467687101
0.12105343490839005
-0.003131068544462323
(667, 512, 300)
(667, 512, 300)
number of surviving frames
569
295
num nan
0
(512, 300)
(512, 300)
/das/work/p17/p17983/cropped_data/scan_data/RuDimerACN_monoscan_0p6ps_011/RuDi

...

SLAAR11-LMOT-M451:ENC_1_BS/pulse_id
SLAAR11-LMOT-M451:ENC_1_BS/pulse_id
passed ID check
numon
666
numoff
334
number of frames
666
334
num nan
0
Izero
0.054733322060549865
0.05498329422775063
0.14087942242622375
0.010902534238994122
(666, 512, 300)
(666, 512, 300)
number of surviving frames
653
321
num nan
0
(512, 300)
(512, 300)
/das/work/p17/p17983/cropped_data/scan_data/RuDimerACN_monoscan_0p6ps_012/RuDimerACN_monoscan_0p6ps_012_step0017.JF02T09V02crop.h5

/sf/alvra/data/p17983/raw/scan_data/RuDimerACN_monoscan_0p6ps_012/RuDimerACN_monoscan_0p6ps_012_step0017.BSREAD.h5
images_roi2
SLAAR11-LMOT-M451:ENC_1_BS/pulse_id
SLAAR11-LMOT-M451:ENC_1_BS/pulse_id
passed ID check
numon
666
numoff
334
number of frames
666
334
num nan
0
Izero
0.05435844059074353
0.05421534318102149
0.1299505978822708
0.01606057770550251
(666, 512, 300)
(666, 512, 300)
number of surviving frames
647
328
num nan
0
(512, 300)
(512, 300)
/das/work/p17/p17983/cropped_data/scan_data/RuDimerACN_monoscan_0p6ps_012/RuDimerA

SLAAR11-LMOT-M451:ENC_1_BS/pulse_id
SLAAR11-LMOT-M451:ENC_1_BS/pulse_id
passed ID check
numon
666
numoff
334
number of frames
666
334
num nan
0
Izero
0.05033603816677978
0.05063799723448629
0.12556353211402893
0.009683635085821152
(666, 512, 300)
(666, 512, 300)
number of surviving frames
642
326
num nan
0
(512, 300)
(512, 300)
/das/work/p17/p17983/cropped_data/scan_data/RuDimerACN_monoscan_0p6ps_012/RuDimerACN_monoscan_0p6ps_012_step0031.JF02T09V02crop.h5

/sf/alvra/data/p17983/raw/scan_data/RuDimerACN_monoscan_0p6ps_012/RuDimerACN_monoscan_0p6ps_012_step0031.BSREAD.h5
images_roi2
SLAAR11-LMOT-M451:ENC_1_BS/pulse_id
SLAAR11-LMOT-M451:ENC_1_BS/pulse_id
passed ID check
numon
667
numoff
333
number of frames
667
333
num nan
0
Izero
0.051686365325171665
0.051843579050092056
0.1348305493593216
0.006708001717925072
(667, 512, 300)
(667, 512, 300)
number of surviving frames
650
325
num nan
0
(512, 300)
(512, 300)
/das/work/p17/p17983/cropped_data/scan_data/RuDimerACN_monoscan_0p6ps_012/RuDime

SLAAR11-LMOT-M451:ENC_1_BS/pulse_id
SLAAR11-LMOT-M451:ENC_1_BS/pulse_id
passed ID check
numon
667
numoff
333
number of frames
667
333
num nan
0
Izero
0.05254426725521997
0.053037994521679815
0.15180373191833496
0.01431695744395256
(667, 512, 300)
(667, 512, 300)
number of surviving frames
651
322
num nan
0
(512, 300)
(512, 300)
/das/work/p17/p17983/cropped_data/scan_data/RuDimerACN_monoscan_0p6ps_012/RuDimerACN_monoscan_0p6ps_012_step0045.JF02T09V02crop.h5

/sf/alvra/data/p17983/raw/scan_data/RuDimerACN_monoscan_0p6ps_012/RuDimerACN_monoscan_0p6ps_012_step0045.BSREAD.h5
images_roi2
SLAAR11-LMOT-M451:ENC_1_BS/pulse_id
SLAAR11-LMOT-M451:ENC_1_BS/pulse_id
passed ID check
numon
666
numoff
334
number of frames
666
334
num nan
0
Izero
0.05221041654442956
0.0515151061677002
0.13095538318157196
0.0053209662437438965
(666, 512, 300)
(666, 512, 300)
number of surviving frames
646
324
num nan
0
(512, 300)
(512, 300)
/das/work/p17/p17983/cropped_data/scan_data/RuDimerACN_monoscan_0p6ps_012/RuDimer

SLAAR11-LMOT-M451:ENC_1_BS/pulse_id
SLAAR11-LMOT-M451:ENC_1_BS/pulse_id
passed ID check
numon
666
numoff
334
number of frames
666
334
num nan
0
Izero
0.052268370071392574
0.051806396281755006
0.1615174412727356
0.012941397726535797
(666, 512, 300)
(666, 512, 300)
number of surviving frames
638
325
num nan
0
(512, 300)
(512, 300)
/das/work/p17/p17983/cropped_data/scan_data/RuDimerACN_monoscan_0p6ps_014/RuDimerACN_monoscan_0p6ps_014_step0004.JF02T09V02crop.h5

/sf/alvra/data/p17983/raw/scan_data/RuDimerACN_monoscan_0p6ps_014/RuDimerACN_monoscan_0p6ps_014_step0004.BSREAD.h5
images_roi2
SLAAR11-LMOT-M451:ENC_1_BS/pulse_id
SLAAR11-LMOT-M451:ENC_1_BS/pulse_id
passed ID check
numon
666
numoff
334
number of frames
666
334
num nan
0
Izero
0.05107818618361019
0.05122803785492
0.1256219446659088
0.009724602103233337
(666, 512, 300)
(666, 512, 300)
number of surviving frames
646
324
num nan
0
(512, 300)
(512, 300)
/das/work/p17/p17983/cropped_data/scan_data/RuDimerACN_monoscan_0p6ps_014/RuDimerACN

OSError: Unable to open file (unable to open file: name = '/das/work/p17/p17983/cropped_data/scan_data/RuDimerACN_monoscan_0p6ps_014/RuDimerACN_monoscan_0p6ps_014_step0006.JF02T09V02crop.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)